In [23]:
import os
import pandas as pd
import numpy as np
import pickle

from tqdm import tqdm

In [25]:
train_path = './data/root/train/'
test_path = './data/root/test/'
train_starttime_path = './data/train_starttime.pkl'
test_starttime_path = './data/test_starttime.pkl'

### 计算小时差

In [6]:
""" time1 = notes_nonull.iloc[2].CHARTTIME
time2 = notes_nonull.iloc[1].CHARTTIME
#diff_time = time1 - time2
diff_time = (time1 - time2)/np.timedelta64(1, 's')/60./60
print(time1, time2)
print(diff_time) """

" time1 = notes_nonull.iloc[2].CHARTTIME\ntime2 = notes_nonull.iloc[1].CHARTTIME\n#diff_time = time1 - time2\ndiff_time = (time1 - time2)/np.timedelta64(1, 's')/60./60\nprint(time1, time2)\nprint(diff_time) "

In [17]:
""" events_3 = pd.read_csv('./data/root/train/3/events.csv')
stays_3 = pd.read_csv('./data/root/train/3/stays.csv')
print(events_3.dtypes)
#print(stays_3.dtypes)
events_3['ICUSTAY_ID'] = events_3['ICUSTAY_ID'].astype(int)
print(events_3.dtypes) """

SUBJECT_ID      int64
HADM_ID         int64
ICUSTAY_ID    float64
CHARTTIME      object
ITEMID          int64
VALUE          object
VALUEUOM       object
dtype: object
SUBJECT_ID     int64
HADM_ID        int64
ICUSTAY_ID     int32
CHARTTIME     object
ITEMID         int64
VALUE         object
VALUEUOM      object
dtype: object


In [30]:
def all_files(file_path):
    all_files = os.listdir(file_path)
    all_folders = list(filter(lambda x: x.isdigit(), all_files))
    return all_folders

def diff(time1, time2):
    # compute time2-time1
    # return difference in hours
    a = np.datetime64(time1)
    b = np.datetime64(time2)
    return (a-b).astype('timedelta64[h]').astype(int)

    

 ## 查看每个文件的各个id类型、
 

In [59]:
def pi_map_start_time(data_path, start_time_path):
    pi_2_start_time = {}
    allfiles = all_files(data_path)
    for pid in tqdm(allfiles):
        #print(pid)
        pid = int(pid)
        events_path = os.path.join(data_path, str(pid), 'events.csv')
        events = pd.read_csv(events_path)
        events['ICUSTAY_ID'] = events['ICUSTAY_ID'].astype(int)
        
        stays_path = os.path.join(data_path, str(pid), 'stays.csv')
        stays = pd.read_csv(stays_path)
        hadm_ids = list(stays.HADM_ID.values)
        intimes = stays.INTIME.values
        
        sliced = events[events.SUBJECT_ID == pid]
        #print(sliced)
        for hindex, hid in enumerate(hadm_ids):
            sliced_i = sliced[sliced.HADM_ID == hid]
            # 找到第hindex次入院时所有指标的测量时间
            #print(sliced_i)
            chart_times = sliced_i['CHARTTIME']
            # 给指标测量时间排序
            chart_times = chart_times.sort_values()
            #print(chart_times)
            # 找到每次入院时间
            intime = intimes[hindex]
            
            # remove intime from charttime
            # 找到进入ICU内的首次指标测量时间
            result = -1
            for t in chart_times:
                #print(f't: {t}')
                #print(diff(t, intime))
                if diff(t, intime) > 1e-6 :
                    result = t
                    break
            name = str(pid) + '_' + str(hindex+1)
            # p_i_2_starttime: 建立某个病人某次住院 与 首次测量的时间的映射
            pi_2_start_time[name] = result

    with open(start_time_path, 'wb') as f:
        pickle.dump(pi_2_start_time, f, pickle.HIGHEST_PROTOCOL)


In [60]:
pi_map_start_time(train_path, train_starttime_path)
pi_map_start_time(test_path, test_starttime_path)

100%|██████████| 3/3 [00:00<00:00, 28.30it/s]


In [61]:
# 先读取root1/train 文件夹的内容，然后测试是否成功
import pickle
f=open(train_starttime_path,'rb')
data=pickle.load(f)
print(data)

{'11_1': '2178-04-16 05:20:00', '17_1': '2134-12-27 15:55:00', '17_2': '2135-05-09 10:45:00', '23_1': '2153-09-03 06:32:00', '23_2': '2157-10-21 03:40:00'}
